In [52]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [53]:
data = pd.read_csv('InputTrain.csv')
label = pd.read_csv("StepOne_LabelTrain.csv")
test = pd.read_csv("InputTest.csv")
data = data.set_index('Index')
label = label.set_index('Index')
test = test.set_index('Index')

In [54]:
X, y = data.drop('House_id', axis=1), label.drop('House_id', axis=1)
testx = test.drop("House_id",axis=1)

In [55]:
X.insert(len(X.columns), "min", data.drop("House_id",axis=1).min(axis=1))
X.insert(len(X.columns), "max", data.drop("House_id",axis=1).max(axis=1))
X.insert(len(X.columns), "std", data.drop("House_id",axis=1).std(axis=1))
X.insert(len(X.columns), "var", data.drop("House_id",axis=1).var(axis=1))
X.insert(len(X.columns), "mean", data.drop("House_id",axis=1).mean(axis=1))
X

,TimeStep_0,TimeStep_1,TimeStep_2,TimeStep_3,TimeStep_4,TimeStep_5,TimeStep_6,TimeStep_7,TimeStep_8,TimeStep_9,...,TimeStep_2155,TimeStep_2156,TimeStep_2157,TimeStep_2158,TimeStep_2159,min,max,std,var,mean
Index,,,,,,,,,,,,,,,,,,,,,
0,180.000000,180.0,180.0,181.0,180.0,180.0,180.000000,181.0,180.0,181.0,...,238.333333,236.0,234.0,234.0,235.0,179.000000,3566.0,346.031985,119738.134353,274.770139
1,2437.000000,2426.0,2148.0,645.0,642.0,642.0,2256.666667,2436.0,2436.0,2212.0,...,236.000000,234.0,234.0,230.0,233.0,180.000000,5444.0,517.594892,267904.472169,464.613580
2,232.000000,232.0,232.5,233.0,233.0,234.0,233.000000,233.0,232.0,234.0,...,181.500000,181.0,180.0,180.0,180.0,178.666667,553.0,52.104176,2714.845137,212.391667
3,180.333333,181.0,180.0,184.0,181.0,180.0,183.000000,184.0,184.0,183.0,...,349.000000,348.5,340.0,346.0,344.0,151.000000,2701.0,267.951248,71797.871555,302.598225
4,344.000000,341.0,341.0,327.0,327.0,318.0,318.000000,313.0,313.0,315.0,...,235.000000,233.0,233.0,235.0,233.0,153.000000,399.0,37.034982,1371.589907,223.516049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10416,275.000000,272.0,270.0,275.0,269.0,267.0,269.000000,268.5,270.0,285.0,...,280.000000,281.0,280.0,280.0,281.0,230.000000,4092.0,282.094349,79577.221781,345.242593
10417,279.000000,283.5,281.0,281.0,277.0,281.5,283.000000,284.5,281.0,281.0,...,230.000000,230.0,230.5,231.0,231.0,228.000000,737.0,61.822732,3822.050191,293.144290
10418,229.000000,229.0,231.0,231.0,230.0,230.0,229.500000,230.0,231.0,231.0,...,231.000000,231.5,231.0,230.5,231.0,228.000000,1156.0,40.864152,1669.878893,256.794059


In [56]:
testx.insert(len(testx.columns), "min", test.drop("House_id",axis=1).min(axis=1))
testx.insert(len(testx.columns), "max", test.drop("House_id",axis=1).max(axis=1))
testx.insert(len(testx.columns), "std", test.drop("House_id",axis=1).std(axis=1))
testx.insert(len(testx.columns), "var", test.drop("House_id",axis=1).var(axis=1))
testx.insert(len(testx.columns), "mean", test.drop("House_id",axis=1).mean(axis=1))
testx

,TimeStep_0,TimeStep_1,TimeStep_2,TimeStep_3,TimeStep_4,TimeStep_5,TimeStep_6,TimeStep_7,TimeStep_8,TimeStep_9,...,TimeStep_2155,TimeStep_2156,TimeStep_2157,TimeStep_2158,TimeStep_2159,min,max,std,var,mean
Index,,,,,,,,,,,,,,,,,,,,,
0,595.000000,595.0,601.0,592.0,592.0,581.0,592.000000,592.000000,579.0,584.5,...,2541.0,2653.000000,2671.0,2671.0,2652.0,117.0,3696.5,451.849066,204167.578807,334.352238
1,2682.666667,2622.0,2629.0,2635.0,2624.0,2635.0,2636.333333,2652.000000,2652.0,2635.5,...,119.5,119.500000,121.0,121.0,121.0,117.0,3506.0,564.755083,318948.304127,497.435648
2,529.000000,534.0,540.0,530.5,527.0,527.0,533.333333,528.000000,528.0,527.0,...,200.0,201.000000,202.0,202.0,198.0,121.0,5802.0,872.159038,760661.387255,869.328318
3,197.500000,197.5,198.0,200.0,200.0,213.0,207.333333,207.000000,206.0,206.0,...,269.0,264.333333,267.0,267.0,272.0,117.0,3399.0,332.342792,110451.731513,303.780401
4,291.500000,285.0,283.0,284.0,284.0,283.0,280.333333,280.333333,278.0,285.0,...,121.0,120.000000,120.0,120.0,121.0,117.0,3624.0,390.979041,152864.610879,421.444907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,1789.000000,1789.000000,1789.0,1789.0,...,1789.0,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,0.000000,0.000000,1789.000000
2484,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,1789.000000,1789.000000,1789.0,1789.0,...,1789.0,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,0.000000,0.000000,1789.000000
2485,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,1789.000000,1789.000000,1789.0,1789.0,...,1789.0,1789.000000,1789.0,1789.0,1789.0,1789.0,1789.0,0.000000,0.000000,1789.000000


In [57]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [58]:
X

array([[2.11907164e-02, 2.11907164e-02, 2.11907164e-02, ...,
        1.49510397e-01, 2.23533587e-02, 6.73865903e-02],
       [4.00773629e-01, 3.98923646e-01, 3.52169526e-01, ...,
        2.23637759e-01, 5.00138472e-02, 1.31693077e-01],
       [2.99360915e-02, 2.99360915e-02, 3.00201816e-02, ...,
        2.25127050e-02, 5.06821886e-04, 4.62568618e-02],
       ...,
       [2.94315506e-02, 2.94315506e-02, 2.97679112e-02, ...,
        1.76562162e-02, 3.11741970e-04, 6.12974752e-02],
       [3.29633367e-02, 3.24587958e-02, 3.24587958e-02, ...,
        8.85632095e-02, 7.84344207e-03, 6.90500476e-02],
       [3.27951564e-02, 3.29633367e-02, 3.23747057e-02, ...,
        4.22458205e-02, 1.78470935e-03, 6.73918961e-02]])

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [60]:
y_train

,Washing Machine,Dishwasher,Tumble Dryer,Microwave,Kettle
Index,,,,,
3850,0,0,0,0,1
9776,0,0,0,0,0
3632,0,0,0,0,0
4599,1,0,0,1,0
6822,0,0,0,1,1
...,...,...,...,...,...
2895,0,0,0,1,1
7813,0,0,0,0,0
905,0,0,0,0,0


In [61]:
dtrain_reg = xgb.DMatrix(X_train, y_train)

dtest_reg = xgb.DMatrix(X_test, y_test)

In [64]:
test_X = xgb.DMatrix(scaler.transform(testx))

In [65]:
params = {"alpha":2,"subsample":0.829,'min_child_weight':10,'gamma':0.1,'max_depth': 4,'eta':0.1,"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evalist = [(dtrain_reg, 'train'), (dtest_reg, 'eval')]
n = 1000
bst = xgb.train(params, dtrain_reg, n, evalist,early_stopping_rounds=10)

C:\Users\moham\AppData\Roaming\Python\Python310\site-packages\xgboost\core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-rmse:0.46911	eval-rmse:0.46953
[1]	train-rmse:0.44226	eval-rmse:0.44310
[2]	train-rmse:0.41897	eval-rmse:0.42041
[3]	train-rmse:0.39879	eval-rmse:0.40092
[4]	train-rmse:0.38156	eval-rmse:0.38437
[5]	train-rmse:0.36687	eval-rmse:0.37044
[6]	train-rmse:0.35426	eval-rmse:0.35861
[7]	train-rmse:0.34327	eval-rmse:0.34845
[8]	train-rmse:0.33403	eval-rmse:0.34016
[9]	train-rmse:0.32626	eval-rmse:0.33326
[10]	train-rmse:0.31951	eval-rmse:0.32737
[11]	train-rmse:0.31355	eval-rmse:0.32222
[12]	train-rmse:0.30842	eval-rmse:0.31770
[13]	train-rmse:0.30407	eval-rmse:0.31413
[14]	train-rmse:0.30037	eval-rmse:0.31114
[15]	train-rmse:0.29697	eval-rmse:0.30854
[16]	train-rmse:0.29390	eval-rmse:0.30638
[17]	train-rmse:0.29129	eval-rmse:0.30472
[18]	train-rmse:0.28874	eval-rmse:0.30290
[19]	train-rmse:0.28655	eval-rmse:0.30156
[20]	train-rmse:0.28442	eval-rmse:0.30030
[21]	train-rmse:0.28269	eval-rmse:0.29939
[22]	train-rmse:0.28107	eval-rmse:0.29863
[23]	train-rmse:0.27937	eval-rmse:0.29783
[2

[194]	train-rmse:0.18292	eval-rmse:0.28146
[195]	train-rmse:0.18264	eval-rmse:0.28141
[196]	train-rmse:0.18231	eval-rmse:0.28138
[197]	train-rmse:0.18206	eval-rmse:0.28138
[198]	train-rmse:0.18172	eval-rmse:0.28139
[199]	train-rmse:0.18143	eval-rmse:0.28130
[200]	train-rmse:0.18117	eval-rmse:0.28128
[201]	train-rmse:0.18089	eval-rmse:0.28128
[202]	train-rmse:0.18064	eval-rmse:0.28125
[203]	train-rmse:0.18038	eval-rmse:0.28127
[204]	train-rmse:0.18002	eval-rmse:0.28124
[205]	train-rmse:0.17967	eval-rmse:0.28116
[206]	train-rmse:0.17937	eval-rmse:0.28111
[207]	train-rmse:0.17904	eval-rmse:0.28109
[208]	train-rmse:0.17878	eval-rmse:0.28104
[209]	train-rmse:0.17844	eval-rmse:0.28104
[210]	train-rmse:0.17808	eval-rmse:0.28100
[211]	train-rmse:0.17783	eval-rmse:0.28095
[212]	train-rmse:0.17750	eval-rmse:0.28094
[213]	train-rmse:0.17720	eval-rmse:0.28094
[214]	train-rmse:0.17698	eval-rmse:0.28090
[215]	train-rmse:0.17663	eval-rmse:0.28090
[216]	train-rmse:0.17626	eval-rmse:0.28089
[217]	train

eval-rmse:0.29891

In [66]:
bst.save_model("modelxgb_3")

In [67]:
bst.predict(test_X)

array([[ 0.35405138,  0.4780917 ,  0.01519536,  0.53076315,  0.8085525 ],
       [ 0.23746653,  0.35059172,  0.1655642 ,  0.33364052,  0.65288067],
       [ 0.32155365,  0.23219566, -0.03076049,  0.56792355,  0.6275411 ],
       ...,
       [-0.06470596,  0.0376321 , -0.01267934, -0.01720925,  0.08350158],
       [-0.06470596,  0.0376321 , -0.01267934, -0.01720925,  0.08350158],
       [-0.06470596,  0.0376321 , -0.01267934, -0.01720925,  0.08350158]],
      dtype=float32)

In [68]:
test_y =bst.predict(test_X)

In [69]:
test_y

array([[ 0.35405138,  0.4780917 ,  0.01519536,  0.53076315,  0.8085525 ],
       [ 0.23746653,  0.35059172,  0.1655642 ,  0.33364052,  0.65288067],
       [ 0.32155365,  0.23219566, -0.03076049,  0.56792355,  0.6275411 ],
       ...,
       [-0.06470596,  0.0376321 , -0.01267934, -0.01720925,  0.08350158],
       [-0.06470596,  0.0376321 , -0.01267934, -0.01720925,  0.08350158],
       [-0.06470596,  0.0376321 , -0.01267934, -0.01720925,  0.08350158]],
      dtype=float32)

In [14]:
results = test_y
# for i in range(5):
#     results[:, i] = (test_y[:,i] > 0.5).astype(int)

NameError: name 'test_y' is not defined

In [22]:
results

array([[0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [70]:
inputes = pd.read_csv('InputTrain.csv')

In [71]:
output_res = pd.DataFrame(test_y, columns=["Washing Machine", "Dishwasher", "Tumble Dryer", "Microwave", "Kettle"])
output_res.insert(0, "Index", inputes["Index"])
output_res.to_csv("./Results20.csv", index=False)